In [1]:
%run etl_tumblr_to_cassandra.ipynb

import pytumblr
import pandas as pd
import json
from zipfile import ZipFile 
import gzip
from pandas.io.json import json_normalize

In [2]:
# Authenticate via OAuth
# We only need the consumer key to call "posts" function
client = pytumblr.TumblrRestClient('nAvaCgNT6dVls4dxKYnWyM1as57L0aSAkSXAayRCPEtNxJSQjr')
total_recursive_count = 1 # number of extra layers after the original blog seed
starting_seed = 'adriwong' 

In [3]:
# Output : Series
# Input : String seed_blog, Series all_sources, Integer recursive_count
# Summary : createSources returns a pandas Series containing blognames that were reblogged by seed_blog without duplicates and NANs
def createSources(seed_blog, all_sources, recursive_count):
    if recursive_count >= total_recursive_count:
        return all_sources
    posts_json = client.posts(seed_blog + '.tumblr.com') #json object with all post information
    if 'posts' in posts_json:
        posts_df = json_normalize(posts_json['posts']) #pandas dataframe
        if 'source_title' in posts_df.columns:
            sources = posts_df['source_title'].dropna().drop_duplicates() #pandas dataframe containing only blognames
            all_sources = all_sources.append(sources)
            for s in sources:
                all_sources = all_sources.append(createSources(s, all_sources, recursive_count + 1)) #recursive call here
    return all_sources.drop_duplicates()

In [4]:
# Output : Dictionary
# Input : Series sources
# Summary : getPostInfo(sources) runs through sources, which is a pandas Series, and returns a dictionay with the blogname as key and necessary data as value
def getPostInfo(sources):
    info_dict = {}
    for blog in sources:
        info_dict[blog] = make_json(blog) # make_json is from etl_tumblr_to_cassandra.ipynb
    return info_dict

In [5]:
all_sources = createSources(starting_seed, pd.Series([]), 0)
s = json.dumps(getPostInfo(all_sources))


studioghifli
theartofanimation
mukuros
inluces
amy-box
sh-izukani
tsuiteru
bekki-chan
akkise
phazn
shadow-kat-13
fuyuuyume
shimizuchan


In [8]:
jsonfilename = 'tumblrdata/tumblrdata1119'
with gzip.open(jsonfilename + ".gz", "wb") as f:
    f.write(bytes(s,"utf-8"))